In [ ]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

In [8]:
#upload your CSV files here
df = pd.read_csv('youtube_vids_1st_pull.csv', index_col=0) 
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,rbRYsAgwPBA,Using IFNULL() on a Google SQL Interview Question,2023-04-19,346,19,0
1,IWo8H8U_Ugk,Sorting and Ranking Data in Python: An Overvie...,2023-04-18,133,4,0
2,2uu0prXMhfU,How to Rank() Your Data In Python Pandas,2023-04-12,621,34,1
3,sF6gngs9myY,38 Data Science Tools For Beginners to Experts,2023-04-05,1675,131,12
4,Y5-eAFnxMjU,Maximizing Profits: Solving the Uber Data Proj...,2023-04-04,240,9,0
...,...,...,...,...,...,...
144,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,2554,15,2
145,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1917,29,0
146,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,1178,21,0
147,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,8156,76,1


In [15]:
def connect_to_db(hostname,dbname,port,username,password):
    try:
        conn = ps.connect(host= hostname,database = dbname, user = username, password = password, port = port)
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn

In [9]:
#create table
def create_table(curr):
    create_table_command = (""" CREATE TABLE IF NOT EXISTS videos(
                            video_id VARCHAR(255) PRIMARY KEY,
                            video_title TEXT NOT NULL,
                            upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                            view_count INTEGER NOT NULL,
                            like_count INTEGER NOT NULL,
                            comment_count INTEGER NOT NULL
                        )""")
    curr.execute(create_table_command)

In [10]:
# updating the database
# handling data for scalability

#check to see if video exists
def update_db(curr,df):
    
    tmp_df = pd.DataFrame(columns = ['video_id','video_title','upload_date','view_count','like_count','comment_count'])
    
    for i, row in df.iterrows():
        if check_if_video_exists(curr,row['video_id']): # if video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will append to the db table
            tmp_df = tmp_df.append(row)
    
    return tmp_df


In [11]:
#check if video exists
def check_if_video_exists(curr,video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query,(video_id,))
    
    return curr.fetchone() is not None

In [12]:
#update row if video exists

def update_row(curr,video_id,video_title,view_count,like_count,comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s""")
    vars_to_update = (video_title,view_count,like_count,comment_count,video_id)
    curr.execute(query,vars_to_update)

In [13]:
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count,comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count,comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [14]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['comment_count'])

In [16]:
hostname = 'xxxxxxxxxxxxxxx.rds.amazonaws.com'
dbname = 'xxxxxxxx'
port = '5432'
username = 'xxxxxxxx'
password = 'xxxxxxxx'
conn = None

conn = connect_to_db(hostname,dbname,port,username,password)

Connected!


In [17]:
curr = conn.cursor()

In [18]:
create_table(curr)

In [ ]:
new_vids_df = update_db(curr,df)

In [21]:
new_vids_df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,rbRYsAgwPBA,Using IFNULL() on a Google SQL Interview Question,2023-04-19,346,19,0
1,IWo8H8U_Ugk,Sorting and Ranking Data in Python: An Overvie...,2023-04-18,133,4,0
54,gtHI672Tlbw,DoorDash Medium Level SQL Interview Question,2022-09-08,758,26,2
55,DmUR2QSNUq8,DoorDash Data Science SQL Interview Question W...,2022-09-07,2652,75,8
56,OLG6_EHMhFk,The One Thing That Keeps Data Scientists Up-to...,2022-08-31,690,53,4
...,...,...,...,...,...,...
144,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,2554,15,2
145,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1917,29,0
146,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,1178,21,0
147,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,8156,76,1


In [22]:
append_from_df_to_db(curr, new_vids_df)

In [ ]:
conn.commit() 